In [ ]:
import os
import subprocess
import zipfile
from tqdm import tqdm  # Hiển thị tiến trình tải

# Các đường dẫn đến file danh sách link tải
MODEL_LINKS_FILE = os.path.join(CONFIG_DIR, "model_links.txt")
VAE_LINKS_FILE = os.path.join(CONFIG_DIR, "vae_links.txt")
CONTROLNET_LINKS_FILE = os.path.join(CONFIG_DIR, "controlnet_links.txt")
CLIP_LINKS_FILE = os.path.join(CONFIG_DIR, "clip_links.txt")
UPSCALE_LINKS_FILE = os.path.join(CONFIG_DIR, "upscale_links.txt")
CUSTOM_NODES_FILE = os.path.join(CONFIG_DIR, "custom_nodes.txt")

# Các thư mục lưu model tải về
MODEL_DIR = os.path.join(BASE_DIR, "models/checkpoints")
VAE_DIR = os.path.join(BASE_DIR, "models/vae")
CONTROLNET_DIR = os.path.join(BASE_DIR, "models/controlnet")
CLIP_DIR = os.path.join(BASE_DIR, "models/clip")
UPSCALE_DIR = os.path.join(BASE_DIR, "models/upscale_models")
CUSTOM_NODES_DIR = os.path.join(BASE_DIR, "custom_nodes")

# === Kiểm tra & tạo thư mục nếu chưa tồn tại ===
for directory in [CONFIG_DIR, MODEL_DIR, VAE_DIR, CONTROLNET_DIR, CLIP_DIR, UPSCALE_DIR, CUSTOM_NODES_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# === Cài đặt ComfyUI từ GitHub ===
def install_comfyui():
    print("\n🔍 Kiểm tra ComfyUI...")
    if not os.path.exists(BASE_DIR):
        print("🚀 Cloning ComfyUI repository...")
        subprocess.run(["git", "clone", GITHUB_URL, BASE_DIR], check=True)
        print("✅ Clone ComfyUI hoàn tất!")
    else:
        print("✅ ComfyUI đã có sẵn!")

    os.chdir(BASE_DIR)

    if UPDATE_COMFY_UI:
        print("\n🔄 Đang cập nhật ComfyUI...")
        subprocess.run(["git", "pull"], check=True)
        print("✅ Cập nhật ComfyUI hoàn tất!")

# === Cài đặt dependencies CUDA ===
def install_cuda_dependencies():
    print("\n🔍 Kiểm tra phiên bản CUDA...")
    cuda_versions = ["cu121", "cu118", "cu117"]
    selected_cuda = None

    for version in cuda_versions:
        test_url = f"https://download.pytorch.org/whl/{version}"
        print(f"🔎 Thử cài đặt với CUDA {version}...")
        result = subprocess.run(["pip", "install", "--extra-index-url", test_url, "torch"], capture_output=True, text=True)
        
        if "ERROR" not in result.stderr:
            selected_cuda = version
            print(f"✅ Chọn phiên bản CUDA: {selected_cuda}")
            break
    else:
        print("⚠️ Không xác định được phiên bản CUDA, dùng mặc định.")

    print(f"🚀 Đang cài đặt dependencies với CUDA {selected_cuda}...")
    subprocess.run([
        "pip", "install", "xformers!=0.0.18", "-r", "requirements.txt",
        "--extra-index-url", f"https://download.pytorch.org/whl/{selected_cuda}"
    ], check=True)
    print("✅ Cài đặt dependencies hoàn tất!")

# === Hàm tải file bằng aria2c ===
def download_files(file_path, output_dir):
    """Tải file từ danh sách link có trong file txt bằng aria2c. Phân tách URL & tên file bằng '@='."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if not os.path.exists(file_path):
        print(f"❌ Không tìm thấy file danh sách: {file_path}")
        return

    with open(file_path, "r") as f:
        links = f.readlines()

    for line in tqdm(links, desc="📥 Đang tải", unit="file"):
        line = line.strip()
        if not line:
            continue

        try:
            url, filename = line.split("@=")  # Phân tách URL và tên file
        except ValueError:
            print(f"❌ Lỗi định dạng dòng: {line}")
            continue

        output_path = os.path.join(output_dir, filename)
        if os.path.exists(output_path):
            print(f"✅ Đã tồn tại: {filename}, bỏ qua...")
        else:
            subprocess.run(["aria2c", "-c", "-x", "16", "-s", "16", "-k", "1M", "-d", output_dir, "-o", filename, url])

# === Hàm tải riêng từng loại model ===
def install_checkpoints():
    download_files(MODEL_LINKS_FILE, MODEL_DIR)

def install_vae():
    download_files(VAE_LINKS_FILE, VAE_DIR)

def install_controlnet():
    download_files(CONTROLNET_LINKS_FILE, CONTROLNET_DIR)

def install_clip():
    download_files(CLIP_LINKS_FILE, CLIP_DIR)

def install_upscale_models():
    download_files(UPSCALE_LINKS_FILE, UPSCALE_DIR)

# === Cài đặt Custom Nodes ===
def install_custom_nodes():
    """Clone Custom Nodes từ GitHub và cài đặt dependencies nếu có."""
    if not os.path.exists(CUSTOM_NODES_FILE):
        print(f"❌ Không tìm thấy file danh sách Custom Nodes: {CUSTOM_NODES_FILE}")
        return

    os.chdir(CUSTOM_NODES_DIR)  # Quay vào thư mục custom_nodes trước khi bắt đầu

    with open(CUSTOM_NODES_FILE, "r") as f:
        nodes = f.readlines()

    for node_url in nodes:
        node_url = node_url.strip()
        if not node_url:
            continue
        
        repo_name = node_url.split("/")[-1]
        node_dir = os.path.join(CUSTOM_NODES_DIR, repo_name)

        if os.path.exists(node_dir):
            print(f"✅ Custom Node '{repo_name}' đã tồn tại, bỏ qua...")
            continue

        print(f"🚀 Đang cài đặt Custom Node: {repo_name}")
        subprocess.run(["git", "clone", node_url, node_dir])

        os.chdir(node_dir)

        req_file = os.path.join(node_dir, "requirements.txt")
        if os.path.exists(req_file):
            print(f"📦 Đang cài đặt dependencies cho {repo_name}...")
            subprocess.run(["pip", "install", "-r", req_file])

        print(f"✅ Cài đặt hoàn tất cho Custom Node: {repo_name}")
        
        os.chdir(CUSTOM_NODES_DIR)

    os.chdir(BASE_DIR)
    print(f"🏠 Quay lại thư mục BASE_DIR: {BASE_DIR}")

# Chạy các quá trình chính